<a href="https://colab.research.google.com/github/w-okada/voice-changer/blob/dev/VoiceRecorder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Voice Recorder
---

このノートでは、MMVCのトレーニング用の音声を録画するアプリ"Voice Recorder"をColab上から起動します。

録音された音声はこのノートを通してGoogle Drive上にアップロードすることができます。

また、従来のVoice Recorderと同様にローカルPCにダウンロードすることもできます。

録音後にブラウザとcolab上のサーバ間でやり取りを行うので、更新に少しタイムラグが発生します。

ご自身のPCでトレーニングを行う予定の場合は、colab上のサーバで録音するメリットはほぼありませんので、より快適な録音をするために[こちらのgithub上のVoice Recorder](https://w-okada.github.io/voice-changer/)をご使用ください。


より詳細な情報はこちらの[リポジトリ](https://github.com/w-okada/voice-changer)からご確認いただけます。


# 録音データを格納するフォルダを指定

フォルダは次の二つを指定する必要があります。
1. 録音アプリ用のキャッシュデータ格納フォルダ
2. トレーニングデータの格納フォルダ

通常、録音データはGoogle Drive上のフォルダに格納すると思います。

まずは(1-1)を実行してドライブをマウントしてください。

その後、(1-2)で上記の格納フォルダを指定してください。

In [1]:
# (1-1) Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# (1-2) 使用するモデルとコンフィグファイルの指定
RECORDER_DATA_DIR="/content/drive/MyDrive/VoiceChanger/voice_data"
MMVC_DATA_DIR="/content/drive/MyDrive/VoiceChanger/dataset"


# リポジトリのクローン
リポジトリをクローンします

In [3]:
# (2) リポジトリのクローン
!git clone https://github.com/w-okada/voice-changer.git
%cd voice-changer/docs/


Cloning into 'voice-changer'...
remote: Enumerating objects: 499, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 499 (delta 26), reused 30 (delta 18), pack-reused 416
Receiving objects: 100% (499/499), 21.10 MiB | 13.43 MiB/s, done.
Resolving deltas: 100% (253/253), done.
/content/voice-changer/docs


# ファイルの配置
アプリケーションの挙動を記した設定ファイルをコピーします(3-1)。(3-2)はコピーした設定ファイルを表示しています。もしかしたらうまく動かないときに役立つかもしれません。

In [4]:
# (3-1) 設定ファイルのコピー
!cp ../template/setting_recorder_colab.json assets/setting.json

In [5]:
# (3-2) 設定ファイルの内容確認

!cat assets/setting.json

{
    "app_title": "voice-recorder",
    "storage_type":"server",
    "use_mel_spectrogram":true,
    "text": [
        {
            "title": "ITA-emotion",
            "wavPrefix": "emotion",
            "file": "./assets/text/ITA_emotion_all.txt",
            "file_hira": "./assets/text/ITA_emotion_all_hira.txt"
        },
        {
            "title": "ITA-recitation",
            "wavPrefix": "recitation",
            "file": "./assets/text/ITA_recitation_all.txt",
            "file_hira": "./assets/text/ITA_recitation_all_hira.txt"
        },
        {
            "title": "wagahaiwa",
            "wavPrefix": "wagahaiwa",
            "file": "./assets/text/wagahaiwa.txt",
            "file_hira": "./assets/text/wagahaiwa_hira.txt"
        }
    ]
}


# モジュールのインストール

必要なモジュールをインストールします。

In [6]:
# (4) 設定ファイルの確認
!pip install flask
!pip install flask_cors


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# サーバの起動

サーバを起動します。(5-1) 

サーバの起動状況を確認します。(5-2) 

このセルは繰り返し実行することになるのでCtrl+Retでセルを実行してください。

アクセスできるようになるまで、数秒かかります。

下記のようなテキストが表示されたら起動完了です。

```
[2022-09-13 22:20:49,936] INFO in recorderServer: START APP
 * Serving Flask app "recorderServer" (lazy loading)
 * Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
 * Debug mode: on
[2022-09-13 22:20:49,946] INFO in _internal:  * Running on http://0.0.0.0:8018/ (Press CTRL+C to quit)
[2022-09-13 22:20:49,947] INFO in _internal:  * Restarting with stat
[2022-09-13 22:20:50,166] INFO in recorderServer: START APP
[2022-09-13 22:20:50,174] WARNING in _internal:  * Debugger is active!
[2022-09-13 22:20:50,200] INFO in _internal:  * Debugger PIN: 334-166-753
```



In [7]:
# (5-1) サーバの起動
PORT=8018
get_ipython().system_raw(f'python3 recorderServer.py {PORT} {RECORDER_DATA_DIR} >foo 2>&1 &')

In [8]:
# (5-2) サーバの起動確認 (Ctrl+Retで実行)
!cat foo

[2022-09-13 22:45:22,054] INFO in recorderServer: START APP
 * Serving Flask app "recorderServer" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
[2022-09-13 22:45:22,062] INFO in _internal:  * Running on http://0.0.0.0:8018/ (Press CTRL+C to quit)
[2022-09-13 22:45:22,063] INFO in _internal:  * Restarting with stat
[2022-09-13 22:45:22,238] INFO in recorderServer: START APP
[2022-09-13 22:45:22,244] WARNING in _internal:  * Debugger is active!
[2022-09-13 22:45:22,268] INFO in _internal:  * Debugger PIN: 334-166-753


# プロキシを起動
ウェブサーバへのアクセスをするためのプロキシを起動します。

表示されたURLをクリックして開くと別タブでアプリが開きます。

Colabなので、ロードにある程度時間がかかります(30秒くらい)。

In [9]:
# (7) プロキシを起動
from google.colab import output

output.serve_kernel_port_as_window(PORT)

<IPython.core.display.Javascript object>

# トレーニング用データフォルダ

以下、トレーニング用のフォルダを作成します。




In [43]:
corpus_id = "14oXoQqLxRkP8NJK8qMYGee1_q2uEED1z"

data_setting = [
    ["user",     "",                                  "",              "00_myvoice",         "107"],
    ["zundamon", "1h8Ajyvoig7Hl3LSSt2vYX0sUHX3JDF3R", "1205_zundamon", "01_target_zundamon", "100"],
    ["tsumugi",  "14zE0F_5ZCQWXf6m6SUPF5Y3gpL6yb7zk", "344_tsumugi",   "02_target_tsumugi",  "103"],
    ["metan",    "1iCrpzhqXm-0YdktOPM8M1pMtgQIDF3r4", "459_methane",   "03_target_metan",    "102"],
    ["sora",     "1MXfMRG_sjbsaLihm7wEASG2PwuCponZF", "912_sora",      "04_target_ksora",    "101"],
]

In [57]:
import os, glob

os.makedirs(MMVC_DATA_DIR, exist_ok=True)
speaker_list = os.path.join(MMVC_DATA_DIR, "multi_speaker_correspondence.txt")
!echo "00_myvoice|107" > {speaker_list}
!echo "01_target_zundamon|100" >> {speaker_list}
!echo "02_target_tsumugi|103" >> {speaker_list}
!echo "03_target_metan|102" >> {speaker_list}
!echo "04_target_ksora|101" >> {speaker_list}

!cat {speaker_list}



00_myvoice|107
01_target_zundamon|100
02_target_tsumugi|103
03_target_metan|102
04_target_ksora|101


In [71]:
!rm -rf /content/drive/MyDrive/VoiceChanger/train_data/00_myvoice/wav/*

In [73]:
import gdown

gdown.download(f'https://drive.google.com/uc?id={corpus_id}', f'ita_corpus.zip', quiet=False)
!unzip -oq 'ita_corpus.zip'

for chara in data_setting:
  chara_root_dir = os.path.join(MMVC_DATA_DIR, chara[3])
  os.makedirs(chara_root_dir, exist_ok=True)
  
  chara_text_dir = os.path.join(chara_root_dir, "text")
  os.makedirs(chara_text_dir, exist_ok=True)
  chara_wav_dir = os.path.join(chara_root_dir, "wav")
  os.makedirs(chara_wav_dir, exist_ok=True)

  if chara[0] != "user":
      gdown.download(f'https://drive.google.com/uc?id={chara[1]}', f'{chara[0]}.zip', quiet=False)
      !unzip -f '{chara[0]}.zip'
      !cp -rf {chara[2]}/* {chara_root_dir}/

  if chara[0] == "user":
    !cp MMVC向けITAコーパス文章ファイル_配布用/ITA_emotion_hira_100file/*    {chara_text_dir}
    !cp MMVC向けITAコーパス文章ファイル_配布用/ITA_recitation_hira_324file/* {chara_text_dir}

    file_list = [os.path.abspath(p) for p in glob.glob(f"{RECORDER_DATA_DIR}/*/*.zip")]
    for f in list(file_list):
      # print(f)
      basename = os.path.basename(f)
      wavname = os.path.splitext(basename)[0] + ".wav"
      full_path = os.path.join(chara_wav_dir, wavname)
      # print(basename, wavname, full_path)
      !unzip -oq {f} vf24kTrim.wav
      !cp vf24kTrim.wav {full_path}






Downloading...
From: https://drive.google.com/uc?id=14oXoQqLxRkP8NJK8qMYGee1_q2uEED1z
To: /content/voice-changer/docs/ita_corpus.zip
100%|██████████| 1.20M/1.20M [00:00<00:00, 87.9MB/s]


/content/drive/MyDrive/VoiceChanger/voice_data/ITA-emotion/emotion000.zip
/content/drive/MyDrive/VoiceChanger/voice_data/ITA-emotion/emotion002.zip
/content/drive/MyDrive/VoiceChanger/voice_data/ITA-emotion/emotion001.zip


Downloading...
From: https://drive.google.com/uc?id=1h8Ajyvoig7Hl3LSSt2vYX0sUHX3JDF3R
To: /content/voice-changer/docs/zundamon.zip
100%|██████████| 55.6M/55.6M [00:00<00:00, 251MB/s]


Archive:  zundamon.zip


Downloading...
From: https://drive.google.com/uc?id=14zE0F_5ZCQWXf6m6SUPF5Y3gpL6yb7zk
To: /content/voice-changer/docs/tsumugi.zip
100%|██████████| 73.0M/73.0M [00:00<00:00, 226MB/s]


Archive:  tsumugi.zip


Downloading...
From: https://drive.google.com/uc?id=1iCrpzhqXm-0YdktOPM8M1pMtgQIDF3r4
To: /content/voice-changer/docs/metan.zip
100%|██████████| 51.8M/51.8M [00:00<00:00, 219MB/s]


Archive:  metan.zip


Downloading...
From: https://drive.google.com/uc?id=1MXfMRG_sjbsaLihm7wEASG2PwuCponZF
To: /content/voice-changer/docs/sora.zip
100%|██████████| 70.2M/70.2M [00:00<00:00, 184MB/s]


Archive:  sora.zip
